In [1]:
from biosimulator_processes.compare import ComparisonDocument   
from process_bigraph import pp

CobraProcess registered successfully.
CopasiProcess registered successfully.
SmoldynProcess not available. Error: 
PLEASE NOTE: Smoldyn is not correctly installed on your system which prevents you from using the SmoldynProcess. Please refer to the README for further information on installing Smoldyn.
TelluriumProcess registered successfully.


In [2]:
simulators = ['copasi', 'tellurium']  # , 'amici']
duration = 30
n_steps = 200
model_filepath = '../biosimulator_processes/model_files/sbml/Caravagna2010.xml'


comparison = ComparisonDocument(simulators, duration, n_steps, model_filepath)

In [3]:
pp(comparison.composite)

{ 'emitter': { '_type': 'step',
               'address': 'local:ram-emitter',
               'config': { 'emit': { 'floating_species_concentrations': 'tree[float]',
                                     'time': 'float'}},
               'inputs': { 'floating_species_concentrations': [ 'floating_species_concentrations_store'],
                           'time': ['time_store']}},
  'processes': { 'copasi': { '_type': 'process',
                             'address': 'local:copasi',
                             'config': { 'model': { 'model_source': '../biosimulator_processes/model_files/sbml/Caravagna2010.xml'}},
                             'inputs': { 'floating_species_concentrations': [ 'floating_species_concentrations_store'],
                                         'model_parameters': [ 'model_parameters_store'],
                                         'reactions': ['reactions_store'],
                                         'time': ['time_store']},
                             

In [ ]:
from process_bigraph import Process, Step, pp




class SimulatorComparison(Process):
    config_schema = {
        'duration': 'number',  
        'simulators': 'list[string]',
        'framework_type': 'maybe[string]'  # ie: 'ode', 'stochastic', "method" types
    }
    
    def __init__(self, config=None, core=None):
        """
        1. create references to simulator processes
        
        """
        super().__init__(config, core)
        
        
 
 
 
 class GeneticParameterOptimizer(Process):
     config_schema = {
         'target_parameter': {  # experimental data
             'name': 'string',
             'target_value': 'float'
         },
         'comparison_results': 'tree[float]'  # sim1Name: values over time, sim2Name: etc
     }
     
     def __init__(self, config=None, core=None):
         super().__init__(config, core)
         
     def calculate_fitness(self, member: float) -> float:
         return self.config['target_parameter']['target_value'] - member